In [22]:
import pandas as pd
import numpy as np

In [59]:
def compute_mean_sector_multiples():
    
    # Load databases
    general_info = pd.read_csv('./databases/general_info.csv')
    last_valuations = pd.read_csv('./databases/last_valuations.csv')
    
    # Select relevant data and merge
    general_info = general_info[['ticker', 'shortName', 'sector', 'industry']]
    last_valuations = pd.merge(general_info, last_valuations, on='ticker')
    last_valuations = last_valuations[['sector', 'priceToBook', 'enterpriseToRevenue', 'enterpriseToEbitda', 'forwardPE', 'trailingPE', 'pegRatio', 'priceToSalesTrailing12Months']]
    
    # Group by sector for test purposes but probably better by industry with a bigger sample
    mean_sector_multiples = multiples.groupby(by='sector').mean()
    
    return mean_sector_multiples

In [124]:
def reformat_valuation():
    
    # Quick&dirty, review databases architecture to have EV, bridge, VT, shares, stock price
    # Check diff stock_price computation with regularMarketPrice (probably fully diluted effect)
    last_valuations = pd.read_csv('./databases/last_valuations.csv')
    stock_info = pd.read_csv('./databases/stock_info.csv')
    
    # Select
    last_valuations = last_valuations[['ticker', 'bookValue', 'enterpriseValue', 'marketCap']]
    stock_info = stock_info[['ticker', 'sharesOutstanding']]
    
    # Join
    last_valuations = pd.merge(last_valuations, stock_info, on='ticker')
    
    # Transform
    last_valuations['bridge_enterpriseValue_marketCap'] = last_valuations['enterpriseValue'] - last_valuations['marketCap']
    last_valuations['stock_price'] = last_valuations['marketCap'] / last_valuations['sharesOutstanding']
    
    # Arrange
    last_valuations = last_valuations[['ticker', 'bookValue', 'enterpriseValue', 'bridge_enterpriseValue_marketCap', 'marketCap', 'sharesOutstanding', 'stock_price']]
    
    return last_valuations

In [115]:
def revenue_valuation(enterpriseToRevenue, totalRevenue, bridge_enterpriseValue_marketCap):
    
    enterpriseValue = enterpriseToRevenue * totalRevenue
    marketCap = enterpriseValue - bridge_enterpriseValue_marketCap
    
    return marketCap

In [116]:
def ebitda_valuation(enterpriseToEbitda, ebitda, bridge_enterpriseValue_marketCap):
    
    enterpriseValue = enterpriseToEbitda * ebitda
    marketCap = enterpriseValue - bridge_enterpriseValue_marketCap
    
    return marketCap

In [117]:
def earnings_valuation(trailingPE, earnings, bridge_enterpriseValue_marketCap):
    
    marketCap = trailingPE * earnings
    enterpriseValue = marketCap + bridge_enterpriseValue_marketCap
    
    return marketCap   

In [118]:
def book_valuation(priceToBook, bookValue, bridge_enterpriseValue_marketCap):
    
    marketCap = priceToBook * bookValue
    enterpriseValue = marketCap + bridge_enterpriseValue_marketCap
    
    return marketCap

In [148]:
def peers_valuation():

    # Load
    financials = pd.read_csv('./databases/financials.csv')
    general_info = pd.read_csv('./databases/general_info.csv')
    mean_sector_multiples = compute_mean_sector_multiples()
    last_valuations = reformat_valuation()

    # Select
    financials = financials[['ticker', 'totalRevenue', 'ebitda', 'freeCashflow']] # Missing earnings for now, used FCF instead just for computation mecanism (but result will be meaningless)
    general_info = general_info[['ticker', 'sector']]
    last_valuations = last_valuations[['ticker', 'bookValue', 'bridge_enterpriseValue_marketCap']]

    # Join
    financials = pd.merge(general_info, financials, on='ticker')
    financials = pd.merge(financials, last_valuations, on='ticker')
    valuation = pd.merge(financials, mean_sector_multiples, on='sector')

    # Apply valuation functions
    valuation['marketCap_revenue'] = valuation.apply(lambda x: revenue_valuation(x['enterpriseToRevenue'], x['totalRevenue'], x['bridge_enterpriseValue_marketCap']), axis=1)
    valuation['marketCap_ebitda'] = valuation.apply(lambda x: ebitda_valuation(x['enterpriseToEbitda'], x['ebitda'], x['bridge_enterpriseValue_marketCap']), axis=1)
    valuation['stock_price_book'] = valuation.apply(lambda x: book_valuation(x['priceToBook'], x['bookValue'], x['bridge_enterpriseValue_marketCap']), axis=1)

    # TBU with earnings and not freeCashflow !!!! ---------------------
    valuation['marketCap_earnings'] = valuation.apply(lambda x: earnings_valuation(x['trailingPE'], x['freeCashflow'], x['bridge_enterpriseValue_marketCap']), axis=1)

    # Final select
    valuation = valuation[['ticker', 'marketCap_revenue', 'marketCap_ebitda', 'marketCap_earnings', 'stock_price_book']]
    
    return valuation

In [155]:
def to_stock_price():
    
    # Load
    valuation = peers_valuation()
    stock_info = pd.read_csv('./databases/stock_info.csv')
    
    # Select
    stock_info = stock_info[['ticker', 'sharesOutstanding']]

    # Join
    valuation = pd.merge(valuation, stock_info, on='ticker')

    # Transform
    valuation['stock_price_revenue'] = valuation['marketCap_revenue'] / valuation['sharesOutstanding']
    valuation['stock_price_ebitda'] = valuation['marketCap_ebitda'] / valuation['sharesOutstanding']
    valuation['stock_price_earnings'] = valuation['marketCap_earnings'] / valuation['sharesOutstanding']
    
    # valuation['mean_stock_price'] = (valuation['stock_price_book'] + valuation['stock_price_revenue'] + valuation['stock_price_ebitda'] + valuation['stock_price_earnings']) / 4
    valuation['mean_stock_price'] = valuation.apply(lambda x: np.mean([x['stock_price_book'], x['stock_price_revenue'], x['stock_price_ebitda'], x['stock_price_earnings']]), axis=1)

    # Final select
    valuation = valuation[['ticker', 'stock_price_book', 'stock_price_revenue', 'stock_price_ebitda', 'stock_price_earnings', 'mean_stock_price']]
    
    return valuation

In [156]:
to_stock_price()

,ticker,stock_price_book,stock_price_revenue,stock_price_ebitda,stock_price_earnings,mean_stock_price
0,ACA.PA,13.430729,49.231301,NaN,NaN,NaN
1,BNP.PA,71.403892,272.910986,NaN,NaN,NaN
2,CS.PA,13.868356,-306.266145,51.997543,16.376448,-56.005949
3,EDEN.PA,-1.996329,-50.479682,44.555921,49.610287,10.422549
4,GLE.PA,59.407223,334.528426,NaN,NaN,NaN
5,AI.PA,91.649557,84.332026,90.913124,51.970453,79.716290
6,MT.AS,121.873424,154.770474,99.708544,117.900041,123.563120
7,AIR.PA,57.932489,133.329837,108.429725,41.608692,85.325186
8,ALO.PA,74.383076,63.587928,20.024850,22.254593,45.062612
9,DG.PA,144.765167,142.813119,148.137518,205.263424,160.244807
